In [ ]:
#importing some libraries

import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

In [ ]:
#downloading the data files

TRAIN_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv"
TEST_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv"

In [ ]:
#testing and training

train_file_path = tf.keras.utils.get_file("train-data.tsv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("valid-data.tsv", TEST_DATA_URL)

df_train = pd.read_csv(train_file_path, sep="\t", header=None, names=['y', 'x'])
df_test = pd.read_csv(test_file_path, sep="\t", header=None, names=['y', 'x'])

In [ ]:
#cleaning thd data_set

def clean_txt(txt):
    txt = re.sub(r'([^\s\w])+', ' ', txt)
    txt = " ".join([lemmatizer.lemmatize(word) for word in txt.split() if not word in stopwords_eng])
    txt = txt.lower()
    return txt


In [ ]:
#preprocessing the text messages

stopwords_eng = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
X_train = df_train['x'].apply(lambda x: clean_txt(x))

max_words = 1000
max_len = 500


In [ ]:
# Creating Tokenizer

t = Tokenizer(num_words=max_words)
t.fit_on_texts(X_train)

sequences = t.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

# Define the model
i = tf.keras.layers.Input(shape=[max_len])
x = tf.keras.layers.Embedding(max_words, 50, input_length=max_len)(i)
x = tf.keras.layers.LSTM(64)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=i, outputs=x)
model.compile(
    loss='binary_crossentropy',
    optimizer='RMSprop',
    metrics=['accuracy']
)


In [ ]:
#training the model

r = model.fit(sequences_matrix, df_train['y'].map({'ham': 0, 'spam': 1}),
              batch_size=256, epochs=10, validation_split=0.02,
              callbacks=[tf.keras.callbacks.EarlyStopping(
                  monitor='val_loss', min_delta=0.0001)])


In [ ]:
def preprocess_messages(messages):
    cleaned_messages = messages.apply(lambda x: clean_txt(x))
    sequences = t.texts_to_sequences(cleaned_messages)
    sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)
    return sequences_matrix

def predict_message(pred_text):
    preprocessed_text = preprocess_messages(pd.Series([pred_text]))
    prediction = model.predict(preprocessed_text)[0][0]
    label = "ham" if prediction < 0.01 else "spam"
    return [prediction, label]


In [ ]:

# Test the predictions
def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"
                    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

test_predictions()


In [ ]:
#testing some more messages

def test_predictions():
    test_messages_list = ["Hello, how are you?",
                          "Congratulations! You won a prize!",
                          "Lahore da pawa Akthar Lava",
                          "Can you pick me up from the station?",
                          "Sure I will pick them up for you."
                          "I'll see you tomorrow.",
                          "URGENT: Your account has been compromised.",
                          "Have a great day!",
                          "You Just Won 100$",
                          "Verify your bank account"]

    for msg in test_messages_list:
        prediction = predict_message(msg)
        print(f"Message: '{msg}' --> Prediction: {prediction[1]} (Likelihood: {prediction[0]:.5f})")

test_predictions()
